In [1]:
import numpy as np
import matplotlib.pyplot as plt

import numpy.polynomial.polynomial as poly
import csv
import sys
import os
import importlib as imp
from   timeit import default_timer as timer
import rebound

import alderaan.io as io
from alderaan.constants import *
from alderaan.utils import *

global_start_time = timer()

In [2]:
PRIMARY_DIR  = '/Users/research/projects/alderaan/'
CSV_FILE = PRIMARY_DIR + "Catalogs/simulated_catalog_eccentric.csv"
SIM_DIR  = PRIMARY_DIR + "Simulations/TTVs/"

# check if all the paths exist and create them if not
if os.path.exists(SIM_DIR) == False:
    os.mkdir(SIM_DIR)

In [3]:
# Read in the data from csv file
print('Reading in data from csv file')

# read in a csv file containing info on targets
csv_keys, csv_values = io.read_csv_file(CSV_FILE)

# put these csv data into a dictionary
catalog = {}
for k in csv_keys: 
    catalog[k] = io.get_csv_data(k, csv_keys, csv_values)
    
k0 = "koi_id"
    
print("Loaded {0} real KOIs".format(len(catalog[k0])))

Reading in data from csv file
Loaded 983 real KOIs


In [4]:
# convert datatypes
for k in catalog.keys():
    try:
        catalog[k] = np.asarray(catalog[k], dtype="float")
    except:
        catalog[k] = np.asarray(catalog[k])
    
    
catalog["npl"] = np.asarray(catalog["npl"], dtype="int")
catalog["kic_id"] = np.asarray(catalog["kic_id"], dtype="int")

In [5]:
TIME_START = 0.
TIME_END = 1600.

# put all epochs in range (TIME_START, period)

for i, koi in enumerate(catalog["koi_id"]):
    while(catalog["epoch"][i] > TIME_START):
        catalog["epoch"][i] -= catalog["period"][i]

    catalog["epoch"][i] += catalog["period"][i]

# Parametric TTVs
### Single planet systems

In [6]:
for i, koi in enumerate(catalog["koi_id"]):
    #print(i, koi, catalog["ttv_type"][i])
    ephemeris = np.arange(catalog["epoch"][i], TIME_END, catalog["period"][i])
    inds = np.arange(len(ephemeris))
        
    
    # linear ttvs
    if catalog["ttv_type"][i] == "linear":
        omc = np.zeros_like(ephemeris)
        tts = ephemeris + omc
        
        
    # quadratic ttvs
    if catalog["ttv_type"][i] == "quadratic":
        x = 2*(ephemeris-TIME_START)/(TIME_END-TIME_START) - 1
        Leg2 = 0.5*(3*x**2 - 1)
        
        scale = np.random.choice([0.1/24, catalog["duration"][i]/24/3])
        C2 = np.random.normal(loc=0, scale=scale)
        
        omc = C2*Leg2
        tts = ephemeris + omc
        
        
    # cubic ttvs
    if catalog["ttv_type"][i] == "cubic":
        x = 2*(ephemeris-TIME_START)/(TIME_END-TIME_START) - 1
        
        Leg2 = 0.5*(3*x**2 - 1)
        Leg3 = 0.5*(5*x**3 - 3*x)
        
        scale = np.random.choice([0.1/24, catalog["duration"][i]/24/3])
        C2, C3  = np.random.normal(loc=0, scale=scale, size=2)
        
        omc = C2*Leg2 + C3*Leg3
        tts = ephemeris + omc
        
        
    # sinusoidal ttvs
    if catalog["ttv_type"][i] == "sinusoidal":
        fmin = 2/(TIME_END - TIME_START)
        fmax = 1/(4*catalog["period"][i])
        logf = np.random.uniform(np.log(fmin), np.log(fmax))
        
        scale = np.random.choice([0.1/24, catalog["duration"][i]/24/3])
        A, B  = np.random.normal(loc=0, scale=scale, size=2)

        f = np.exp(logf)
        t = np.copy(ephemeris)
        
        omc = A*np.sin(2*pi*f*t) + B*np.cos(2*pi*f*t)
        tts = ephemeris + omc        
    
    
    # gaussian (white noise) ttvs
    if catalog["ttv_type"][i] == "gaussian":
        scale = np.random.choice([0.1/24, catalog["duration"][i]/24/3])
        
        omc = np.random.normal(loc=0, scale=scale, size=len(ephemeris))
        tts = ephemeris + omc
        
        
    # ultra-short period TTVs treated as linear
    if catalog["ttv_type"][i] == "usp":
        omc = np.zeros_like(ephemeris)
        tts = ephemeris + omc
    
    
    # grazing TTVs also treated as linear
    if catalog["ttv_type"][i] == "grazing":
        omc = np.zeros_like(ephemeris)
        tts = ephemeris + omc
        
        
    # save the results
    if (catalog["npl"][i] == 1):
        data_out  = np.vstack([inds, tts]).swapaxes(0,1)
        fname_out = SIM_DIR + "S" + koi[1:] + '_00_sim_ttvs.txt'

        np.savetxt(fname_out, data_out, fmt=('%1d', '%.8f'), delimiter='\t')

### 2-planet systems

In [7]:
eccentric_doubles = np.unique(catalog["koi_id"][catalog["ttv_type"] == "eccentric"])

ecc_new = []
omega_new = []

for i, koi in enumerate(eccentric_doubles):
    use = catalog["koi_id"] == koi
    npl = catalog["npl"][use][0]
    
    per = catalog["period"][use]
    t0 = catalog["epoch"][use]
    
    for j in range(npl):
        ephemeris = np.arange(t0[j], TIME_END, per[j])
        inds = np.arange(len(ephemeris))
    
        data_out  = np.vstack([inds, ephemeris]).swapaxes(0,1)
        fname_out = SIM_DIR + "S" + koi[1:] + '_{:02d}'.format(j) + '_sim_ttvs.txt'

        np.savetxt(fname_out, data_out, fmt=('%1d', '%.8f'), delimiter='\t')

# N-body simulations with REBOUND
### Multiplanet systems

In [8]:
def run_simulation(Mstar, mass, per, ecc, omega, mean_anomaly):
    """
    Docstring
    """
    
    NPL = len(mass)
    sma = get_sma(per, Mstar)*RSAU
    
    
    # set up simulation
    sim = rebound.Simulation()
    sim.units = ("AU", "Msun", "days")
    sim.add(m=Mstar)

    for npl in range(NPL):
        sim.add(m=mass[npl], a=sma[npl], e=ecc[npl], omega=omega[npl], M=mean_anomaly[npl])

    p = sim.particles
    
                                                                 
    # do the integration
    transit_times = []

    for npl in range(NPL):
        transit_times.append([])

        while sim.t < 1600:
            y_old = p[npl+1].y - p[0].y

            t_old = sim.t
            sim.integrate(sim.t + per.min()/20)
            t_new = sim.t

            # sign of y changes and planet is in front of star (x > 0)
            if y_old*(p[npl+1].y-p[0].y) < 0 and p[npl+1].x-p[0].x > 0:

                # bisect until precision is reached
                while t_new-t_old > 1e-7:
                    if y_old*(p[npl+1].y-p[0].y) < 0:
                        t_new = sim.t
                    else:
                        t_old = sim.t
                    sim.integrate( (t_new+t_old)/2)

                transit_times[npl].append(sim.t)

               # integrate 0.05 to be past the transit        
                sim.integrate(sim.t + per.min()/20)

        sim.integrate(0.)
        
    return transit_times

In [9]:
rebound_systems = np.unique(catalog["koi_id"][catalog["ttv_type"] == "rebound"])

ecc_new = []
omega_new = []

for i, koi in enumerate(rebound_systems):
    print(i, koi)
    
    # get parameters from simulated catalog
    use = catalog["koi_id"] == koi
    
    npl    = catalog["npl"][use][0]
    Mstar  = catalog["mstar"][use][0]
    mass   = catalog["pmass"][use]/MSME
    per    = catalog["period"][use]
    M_anom = np.random.uniform(0, 2*pi, npl)
    
    
    # draw eccentricity vectors (see Lithwick, Xie, & Wu 2012)
    ecc = np.ones(npl)
    while np.any(ecc > 0.4):
        esinw, ecosw = np.random.normal(loc=0, scale=0.008, size=2*npl).reshape((2,npl))

        ecc = np.sqrt(esinw**2 + ecosw**2)
        omega = np.arctan2(esinw, ecosw)
    
    
    for e0 in ecc:
        ecc_new.append(e0)
    for w0 in omega:
        omega_new.append(w0)
        
    
    # integrate REBOUND to get transit times
    transit_times = run_simulation(Mstar, mass, per, ecc, omega, M_anom)    
    
    transit_inds  = []
    for j in range(npl):
        transit_inds.append(np.arange(len(transit_times[j])))
        
        
    # save the results
    for j in range(npl):
        data_out  = np.vstack([transit_inds[j], transit_times[j]]).swapaxes(0,1)
        fname_out = SIM_DIR + "S" + koi[1:] + '_{:02d}'.format(j) + '_sim_ttvs.txt'

        np.savetxt(fname_out, data_out, fmt=('%1d', '%.8f'), delimiter='\t')
        

# update eccentricity vectors in the catalog
replace = catalog["ttv_type"] == "rebound"

catalog["ecc"][replace] = ecc_new
catalog["omega"][replace] = omega_new

0 K00041
1 K00116
2 K00172
3 K00216
4 K00222
5 K00244
6 K00253
7 K00260
8 K00266
9 K00270
10 K00271
11 K00282
12 K00283
13 K00284
14 K00298
15 K00301
16 K00304
17 K00307
18 K00313
19 K00314
20 K00316
21 K00317
22 K00337
23 K00354
24 K00386
25 K00413
26 K00416
27 K00427
28 K00430
29 K00459
30 K00471
31 K00474
32 K00475
33 K00508
34 K00520
35 K00521
36 K00528
37 K00542
38 K00546
39 K00547
40 K00548
41 K00551
42 K00566
43 K00567
44 K00574
45 K00581
46 K00582
47 K00584
48 K00590
49 K00623
50 K00626
51 K00638
52 K00645
53 K00647
54 K00654
55 K00679
56 K00693
57 K00700
58 K00708
59 K00730
60 K00734
61 K00736
62 K00759
63 K00775
64 K00784
65 K00790
66 K00806
67 K00829
68 K00842
69 K00870
70 K00877
71 K00886
72 K00896
73 K00912
74 K00934
75 K00935
76 K00951
77 K00988
78 K00999
79 K01101
80 K01106
81 K01113
82 K01148
83 K01236
84 K01270
85 K01276
86 K01366
87 K01404
88 K01426
89 K01435
90 K01519
91 K01522
92 K01529
93 K01534
94 K01593
95 K01598
96 K01613
97 K01616
98 K01707
99 K01751
100 K01788

# Update periods and epochs to least squares fits

In [10]:
epoch_lsq = []
period_lsq = []


for i, koi in enumerate(catalog["koi_id"]):
    use = catalog["koi_id"] == koi
    npl = np.sum(use)
    
    # this if statement avoids double counting multiplant systems
    if len(period_lsq) <= i:
        for j in range(npl):
            data_in = np.loadtxt(SIM_DIR + "S" + koi[1:] + '_{:02d}'.format(j) + '_sim_ttvs.txt')
            inds, tts = np.atleast_2d(data_in).swapaxes(0,1)

            if len(tts) > 1:
                pfit = poly.polyfit(inds, tts, 1)

                epoch_lsq.append(pfit[0])
                period_lsq.append(pfit[1])

            else:
                epoch_lsq.append(tts[0])
                period_lsq.append(catalog["period"][use][j])
                

catalog["epoch"] = np.asarray(epoch_lsq)
catalog["period"] = np.asarray(period_lsq)

# Update transit durations

In [11]:
# recalculate transit durations
sma = get_sma(catalog["period"], catalog["mstar"])

catalog["duration"] = 24*get_dur_tot(catalog["period"], 
                                     catalog["prad"]/RSRE, 
                                     catalog["rstar"],
                                     catalog["impact"],
                                     sma,
                                     catalog["ecc"],
                                     catalog["omega"])

# Do some cleanup

In [12]:
all_keys = list(catalog.keys())
int_keys = ['kic_id', 'npl', 'depth']
string_keys = ['planet_name', 'disposition', 'koi_id', 'ttv_type']
precise_keys = ['period', 'epoch']


for k in catalog.keys():
    if np.isin(k, int_keys):
        catalog[k] = np.array(catalog[k], dtype="int")
    elif np.isin(k, string_keys):
        catalog[k] = catalog[k]
    elif np.isin(k, precise_keys):
        catalog[k] = np.round(np.array(catalog[k], dtype="float"), 5)
    else:
        catalog[k] = np.round(np.array(catalog[k], dtype="float"), 3)

# Write out catalog

In [13]:
WRITENEW = True
if WRITENEW:
    with open(CSV_FILE, "w") as outfile:
        writer = csv.writer(outfile)
        writer.writerow(catalog.keys())
        writer.writerows(zip(*catalog.values()))

In [14]:
print('TOTAL RUNTIME = %.2f min' %((timer()-global_start_time)/60))

TOTAL RUNTIME = 2.68 min
